In [1]:
import numpy as np
from alc_M import*

In [2]:
###----------EJERCICIO 1-------------------

def cargarDataset():
    base = "template-alumnos/dataset/cats_and_dogs"

    # --- TRAIN ---
    X_cats = np.load(f"{base}/train/cats/efficientnet_b3_embeddings.npy")   
    X_dogs = np.load(f"{base}/train/dogs/efficientnet_b3_embeddings.npy")   

    X_train = np.hstack([X_cats, X_dogs])  

    Y_train = np.hstack([
        np.tile([[1],[0]], (1, X_cats.shape[1])),  # gatos
        np.tile([[0],[1]], (1, X_dogs.shape[1]))   # perros
    ])                                             

    # --- VAL ---
    V_cats = np.load(f"{base}/val/cats/efficientnet_b3_embeddings.npy")
    V_dogs = np.load(f"{base}/val/dogs/efficientnet_b3_embeddings.npy")

    X_val = np.hstack([V_cats, V_dogs])
    Y_val = np.hstack([
        np.tile([[1],[0]], (1, V_cats.shape[1])),
        np.tile([[0],[1]], (1, V_dogs.shape[1]))
    ])

    print("X_train:", X_train.shape, "Y_train:", Y_train.shape)
    print("X_val:", X_val.shape, "Y_val:", Y_val.shape)

    return X_train, Y_train, X_val, Y_val

In [18]:
Xt, Yt, Xv, Yv = cargarDataset()


X_train: (1536, 2000) Y_train: (2, 2000)
X_val: (1536, 1000) Y_val: (2, 1000)


In [ ]:
## FUNCIONES PARA ELMODULO ALC / AUXILIARES


## SUSTITUCION HACIA ADELANTE
def sust_adelante(L,b):

    L = np.array(L, dtype=float)
    b = np.array(b, dtype=float)

    n = L.shape[0]
    x = np.zeros(n)

    for i in range(n):

        suma = 0
        for j in range(i):

            suma += (L[i][j]*x[j])
            
        termino = b[i] - suma
        x[i] = termino/L[i][i]

    return x

## SUSTITUCION HACIA ATRAS
def sust_atras(L, Z):

    L = np.array(L, dtype=float)
    Z = np.array(Z, dtype=float)
    
    n = L.shape[0]
    x = np.zeros(n)



    for i in range(n-1, -1, -1):
        suma = 0
        for j in range(i+1, n):

            suma += (L[i][j]*x[j])
            
        termino = Z[i] - suma

        x[i]=termino/L[i][i]


    return x


def sust_atras_matriz(U, B):
    """
    Resuelve U X = B para cuando B es matriz
   
    """

    U = np.array(U, dtype=float)
    B = np.array(B, dtype=float)

    n, m = B.shape
    X = np.zeros((n, m))

    for j in range(m):
        bj = B[:, j]
        xj = sust_atras(U, bj)
        X[:, j] = xj

    return X

def sust_adelante_matriz(L, B):
    """
    Resuelve L Z = B para cuando B es matriz

    """
    L = np.array(L, dtype=float)
    B = np.array(B, dtype=float)



    n, m = B.shape
    Z = np.zeros((n, m))

    for j in range(m):
        bj = B[:, j]
        zj = sust_adelante(L, bj)
        Z[:, j] = zj

    return Z



In [ ]:
def Cholesky(A):
    
    A = A.astype(float)
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        for j in range(i + 1):

            if i == j:
                suma_cuadrados = 0.0
                for k in range(j):
                    suma_cuadrados += L[i, k] ** 2

                termino = A[i, i] - suma_cuadrados

                if termino <= 0:
                    raise ValueError("Matriz no definida positiva o error numérico")

                L[i, i] = np.sqrt(termino)

            else:
                producto_punto = 0.0
                for k in range(j):
                    producto_punto += L[i, k] * L[j, k]

                L[i, j] = (A[i, j] - producto_punto) / L[j, j]

    return L

def algoritmo1(X,Y):

    X = np.array(X)
    Y = np.array(Y)


    Xt = transpuesta(X)
    

    xxt = X @ Xt

    L = Cholesky(xxt) 

    W = pinvEcuacionesNormales(L,X,Y)

    return W



def pinvEcuacionesNormales(L, X, Y):
    """
    L: (d x d), tal que X X^T = L L^T
    X: (d x p)
    Y: (2 x p)
    Devuelve W: (2 x d)
    """

    # 1) L Z = X
    Z = sust_adelante_matriz(L, X)      # Z: (d x p)

    # 2) L^T V = Z
    Lt = transpuesta(L)
    V = sust_atras_matriz(Lt, Z)        # V: (d x p)

    # 3) W = Y V^T
    Vt = transpuesta(V)                 # (p x d)
    W  = Y @ Vt                         # (2 x d)

    return W

In [8]:
W_cholesky = np.load("W_cholesky.npy")

In [ ]:
## EJERCICIO 4

def reducir_QR(Q, R):
    
    # número de columnas efectivas de X^T
    p = R.shape[1]        # debe ser 1536
    # R puede tener más filas, pero sólo las primeras p son útiles
    Q_red = Q[:, :p]      # (2000 × 1536)
    R_red = R[:p, :p]     # (1536 × 1536)

    return Q_red, R_red

# ===============================================================

# Uso una misma función para calcular W independientemente del método de la factorización

def calcular_W(Q, R, Y):
    # Paso 1: reducir Q y R si tienen filas extra
    Q_red, R_red = reducir_QR(Q, R)

    # Paso 2: queremos resolver V * R^T = Q_red
    # => V = Q_red @ inv(R^T)
    inv_RT = np.linalg.inv(R_red.T)
    V = Q_red @ inv_RT

    # Paso 3: W = Y @ V
    W = Y @ V
    return W

def pinvHouseHolder(Q, R, Y):
     return calcular_W(Q, R, Y)

def pinvGramSchmidt(Q, R, Y):
     return calcular_W(Q, R, Y)

    


In [ ]:
## EJERCICIO 5
def esPseudoInverda(X, pX, tol=1e-8):

   ## Nos fijamo que se cumpla las cuatro condiciones para que sea pseudo inveesa

   return matricesIguales(matmul_multiple(X,pX,X),X,tol) and matricesIguales(matmul_multiple(pX,X,pX),pX,tol) and simetrica(multiplicar(X,pX),tol) and simetrica(multiplicar(pX,X),tol)   

In [15]:
##EJERCICIO 6

def matrizDeConfusion(Xv, W, Yv):
    
    y_pred = W @ Xv     # (2, p)

    M = np.zeros((2,2), dtype=int)
    p = Xv.shape[1]

    for j in range(p):
        # ----- Clase real -----
        if Yv[0, j] == 1:
            real = 0
        else:
            real = 1

        # ----- Clase predicha -----
        if y_pred[0, j] > y_pred[1, j]:
            pred = 0
        else:
            pred = 1

        # ----- Actualizo matriz -----
        M[real, pred] += 1

    total = Yv.shape[1]

    exactitud = (M[0,0] + M[1,1]) / total

    
    precision_0 = M[0,0] / M[0,0] + M[1,0]


    
    precision_1 = M[1,1] / M[1,1] + M[0,1]

    print("Matriz de confusion:\n", M)
    print("Exactitud:", exactitud)
    print("Precision clase 0:", precision_0)
    print("Precision clase 1:", precision_1)

    return M, exactitud, precision_0, precision_1



In [17]:
W_GS = np.load("W_GS.npy")
W_HH = np.load("W_HH.npy")

print("Matriz de confusion usando Gram-Schmidt:")
print(matrizDeConfusion(Xv,W_GS,Yv))

print("Matriz de confusion usando HouseHolder:")

print(matrizDeConfusion(Xv,W_HH,Yv))

print("Matriz de confusion usando Cholesky:")


matrizDeConfusion(Xv,W_cholesky,Yv)


Matriz de confusion usando Gram-Schmidt:
Matriz de confusion:
 [[334 166]
 [150 350]]
Exactitud: 0.684
Precision clase 0: 151.0
Precision clase 1: 167.0
(array([[334, 166],
       [150, 350]]), np.float64(0.684), np.float64(151.0), np.float64(167.0))
Matriz de confusion usando HouseHolder:
Matriz de confusion:
 [[334 166]
 [150 350]]
Exactitud: 0.684
Precision clase 0: 151.0
Precision clase 1: 167.0
(array([[334, 166],
       [150, 350]]), np.float64(0.684), np.float64(151.0), np.float64(167.0))
Matriz de confusion usando Cholesky:
Matriz de confusion:
 [[334 166]
 [150 350]]
Exactitud: 0.684
Precision clase 0: 151.0
Precision clase 1: 167.0


(array([[334, 166],
        [150, 350]]),
 np.float64(0.684),
 np.float64(151.0),
 np.float64(167.0))